In [ ]:
from gridfs import Database
import mysql.connector
import csv
currentDirectory = ""


def getConn():
    dataBase = mysql.connector.connect(
        host="localhost",
        user="root",
        passwd=""
    )
    return dataBase


def mkdir(name, parent):
    # mkdir: create an entry in the metadata table with the parent as the current directory and attributes as dir.
    dataBase = getConn()
    mycursor = dataBase.cursor()
    insert = "INSERT INTO project1.metadata(dir_file_name,parent,file_dir_flag) VALUES (%s,%s,%s)"
    id = 1
    values = (name, parent, int(1))
    mycursor.execute(insert, values)
    id = mycursor.lastrowid
    dataBase.commit()
    return id


def ls():
    # ls: select the entries from the metadata where the parent directory is the current working directory.
    dataBase = getConn()
    query = "SELECT * FROM project1.metadata WHERE parent = %s"
    mycursor = dataBase.cursor()
    values = (currentDirectory,)
    mycursor.execute(query, values)
    myresult = mycursor.fetchall()

    for x in myresult:
        print(x[1])


def rm(file):
    # rm: first retrieve the partition information of the file based on its ID; then delete all the tables of the corresponding partition.

    dataBase = getConn()

    myresult = getPartitionLocations(file)
    mycursor = dataBase.cursor()
    fileId = myresult[0][0]
    for x in myresult:
        table = x[2].split("Table ")[1]
        query = "DROP TABLE project1."+table
        mycursor.execute(query)

    query = "DELETE FROM project1.partition WHERE file_id = %s"
    values = (fileId,)
    mycursor.execute(query, values)
    myresult = mycursor.fetchall()
    dataBase.commit()
    print("Partition removed")


def cat(file):
    # cat: first retrieve the partition information of the file based on its ID; then load each table of the corresponding partition.
    part = getPartitionLocations(file)

    for p in part:
        rows = readPartition(file, p[0])
        for row in rows:
            print(row)


def put(numEntries, fileName):
    # put: first create a new entry in the metadata table of the file;
    # then create entries in the partition table based on the number of partitions the user wants.
    # Finally partition the file into different tables with the name specified in the partition table.
    fileId = mkdir(fileName, currentDirectory)
    dataBase = getConn()
    mycursor = dataBase.cursor()
    cnt = 1
    firstPartId = 0
    for i in range(numEntries):
        insert = "INSERT INTO project1.partition (file_id, `partition`) VALUES(%s,%s)"

        values = (int(fileId), "Table movies_part"+str(cnt))
        cnt = cnt+1
        print(insert)
        print(values)
        mycursor.execute(insert, values)
        dataBase.commit()
        partId = mycursor.lastrowid
        if (firstPartId == 0):
            firstPartId = partId

    importFile(firstPartId, fileName, numEntries)


def importFile(tableNum, fn, partitions):
    rows = []
    fields = []

    with open(fn, 'r', encoding="utf8") as csvfile:
        csvreader = csv.reader(csvfile)

        fields = next(csvreader)
        for row in csvreader:
            movie = [row[5], row[8]]
            rows.append(movie)

        rowCt = len(rows)
        print("rows:", rowCt)
        rowsPerTable = rowCt/partitions
        print("rows:", rowCt, partitions, rowsPerTable)
        test = 0

    createTable(tableNum)
    database = getConn()
    mycursor = database.cursor()
    rowCt = 0
    for row in rows:
        insert = "INSERT INTO project1.movies_part"+str(tableNum)
        insert = insert + "(`id`,`movie_id`, `movie_name`)VALUES (%s,%s,%s)"
        values = (int(tableNum), row[0], row[1])
        mycursor.execute(insert, values)
        database.commit()
        rowCt = rowCt+1
        if rowCt >= rowsPerTable:
            tableNum = tableNum+1
            rowCt = 0
            createTable(tableNum)
        # print(row)


def createTable(idx):
    temp = "CREATE TABLE project1.movies_part" + str(idx)
    temp = temp + \
        "(`id` int(11) NOT NULL , `movie_id` int(11) DEFAULT NULL, `movie_name` varchar(255) DEFAULT NULL)"
    dataBase = getConn()
    mycursor = dataBase.cursor()
    print(temp)
    mycursor.execute(temp)
    dataBase.commit()


def getPartitionLocations(file):
    # getPartitionLocations(file): first get file ID from the metadata table and then use the file ID in the partition table to get the partitions of the file.
    dataBase = getConn()
    mycursor = dataBase.cursor()
    query = "SELECT id FROM project1.metadata WHERE dir_file_name = %s"
    values = (file,)
    mycursor.execute(query, values)
    row = mycursor.fetchone()
    fileId = row[0]

    query = "SELECT * FROM project1.partition WHERE file_id = %s"
    values = (fileId,)
    mycursor.execute(query, values)
    rows = mycursor.fetchall()
    return rows


def readPartition(file, partitionNum):
    # readPartition(file, partition#): load the corresponding table of the partition of the file.
    dataBase = getConn()
    mycursor = dataBase.cursor()
    query = "SELECT id FROM project1.metadata WHERE dir_file_name = %s"
    values = (file,)
    mycursor.execute(query, values)
    row = mycursor.fetchone()
    fileId = row[0]
    query = "SELECT `partition` FROM project1.partition WHERE file_id = %s AND id = %s"
    values = (int(fileId), int(partitionNum))
    mycursor.execute(query, values)
    rows = mycursor.fetchone()
    table = rows[0].split("Table ")[1]
    print("Table: ", table)
    query = "SELECT * FROM project1." + table

    mycursor.execute(query)

    rows = mycursor.fetchall()
    return rows


def start():
    command = ""
    while command != "exit":
        command = input("$")
        if command == "exit":
            return
        args = command.split(" ")
        if args[0] == "put":
            put(int(args[1]), args[2])
        elif args[0] == "li":
            ls()
        elif args[0] == "cat":
            cat(args[1])
        elif args[0] == "rm":
            rm(args[1])


start()